In [1]:
import pytoon

def rasterize(rgba_Dt, xdim=(-1,1,20), ydim=(-1,1,20), pixel_aspect_ratio=1):
    # rgba_Dt should be a function of x and y that returns a 2-tuple of rgbs and Dt, where
    # where Dt is the time resolution to use at that point in space and
    # rgba is a function of time (and it can ignore the time argument), returning a 2-tuple
    # which is the rgb code (in #RRGGBB form) and alpha value at that x,y point (at that time);
    # the code can recover if the alpha value is omitted (a value instead of a tuple is returned)
    # by omitting the alpha channel (no transparency).
    #
    Py = pixel_aspect_ratio
    Px = 1 / Py            # hard-coded:  pixel area is 1
    px = (1.001) * Px/2    # fudge factor to overlap edges just slightly ...
    py = (1.001) * Py/2    # ... looks a lot better than leaving even a tiny gap (due to roundings?)
    pixel = pytoon.polygon(points=[(-px,-py),(px,-py),(px,py),(-px,py)], lstyle=False)
    #
    def color(x, y, layer):
        rgba, Dt = rgba_Dt(x,y)
        if layer=="upper":
            alpha = lambda a: a/2
        else:
            alpha = lambda a: a/(2-a)
        def value(_t_):
            rgb_a = rgba(_t_)
            try:
                rgb, a = rgb_a
            except TypeError:
                rgb, a = rgb_a, 1
            a = 1 if (a is None) else a
            return pytoon.colordef(rgb=rgb, a=alpha(a))
        return pytoon.animated(value, Dt=Dt)
    #
    xmin, xmax, Nx = xdim
    ymin, ymax, Ny = ydim
    Dx = (xmax-xmin) / Nx
    Dy = (ymax-ymin) / Ny
    lower = []
    for i in range(Nx+1):
        x = xmin + i*Dx
        for j in range(Ny+1):
            y = ymin + j*Dy
            lower += [ pixel(fstyle=color(x,y,"upper")).T(i*Px, j*Py) ]
    upper = []
    for i in range(Nx):
        x = xmin + (i+1/2)*Dx 
        for j in range(Ny):
            y = ymin + (j+1/2)*Dy
            upper += [pixel(fstyle=color(x,y,"upper")).T((i+1/2)*Px, (j+1/2)*Py)]
    #
    return pytoon.composite([*lower, *upper])

In [2]:
from math import exp, sqrt, cos, sin, pi, atan2
from util import int_round

red    = 0xcc, 0x33, 0x33
orange = 0xcc, 0x55, 0
yellow = 0x99, 0x88, 0
green  = 0,    0x77, 0x11
blue   = 0x66, 0x77, 0xff
purple = 0xbb, 0x11, 0xee

def map_interval(x):
    return x
    return sin(pi*x/2)**2

def color_map(phi):
    phi /= pi
    if   phi<-2/3:
        w = 3*(phi + 1)
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = red
        r2, g2, b2 = orange
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi<-1/3:
        w = 3*(phi + 2/3)
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = orange
        r2, g2, b2 = yellow
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 0:
        w = 3*(phi + 1/3)
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = yellow
        r2, g2, b2 = green
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 1/3:
        w = 3*phi
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = green
        r2, g2, b2 = blue
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    elif phi< 2/3:
        w = 3*(phi - 1/3)
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = blue
        r2, g2, b2 = purple
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    else:
        w = 3*(phi - 2/3)
        w = map_interval(w)
        v = 1 - w
        r1, g1, b1 = purple
        r2, g2, b2 = red
        rr = int_round(v*r1 + w*r2)
        gg = int_round(v*g1 + w*g2)
        bb = int_round(v*b1 + w*b2)
    return "#{:02x}{:02x}{:02x}".format(rr,gg,bb)

def F(x,y):
    phi = atan2(y,x)
    rgb = color_map(phi)
    alpha = x**2 + y**2
    if alpha>1: alpha = 0
    def rgba(t):
        return rgb, alpha
    return rgba, 0.05


pytoon.composite([
    rasterize(F, xdim=(-1,1,30), ydim=(-1,1,30)),
    pytoon.circle(radius=15, center=(15,15), lstyle=1.5)
]).svg("outputs/rasterized", background="#333333")

[link to animation file](files/outputs/rasterized.svg)